In [1]:
import pandas as pd
import numpy as np

In [2]:
df_adm = pd.read_csv('/home/dc925/project/data/graphmimic/MIMIC/ADMISSIONS.csv')

In [3]:
df_adm.ADMITTIME = pd.to_datetime(df_adm.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_adm.DISCHTIME = pd.to_datetime(df_adm.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_adm.DEATHTIME = pd.to_datetime(df_adm.DEATHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

In [4]:
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])
df_adm = df_adm.reset_index(drop = True)
df_adm['NEXT_ADMITTIME'] = df_adm.groupby('SUBJECT_ID').ADMITTIME.shift(-1)
df_adm['NEXT_ADMISSION_TYPE'] = df_adm.groupby('SUBJECT_ID').ADMISSION_TYPE.shift(-1)

In [5]:
rows = df_adm.NEXT_ADMISSION_TYPE == 'ELECTIVE'
df_adm.loc[rows,'NEXT_ADMITTIME'] = pd.NaT
df_adm.loc[rows,'NEXT_ADMISSION_TYPE'] = np.NaN

In [6]:
df_adm = df_adm.sort_values(['SUBJECT_ID','ADMITTIME'])


In [7]:
df_adm[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']] = df_adm.groupby(['SUBJECT_ID'])[['NEXT_ADMITTIME','NEXT_ADMISSION_TYPE']].fillna(method = 'bfill')
df_adm['DAYS_NEXT_ADMIT']=  (df_adm.NEXT_ADMITTIME - df_adm.DISCHTIME).dt.total_seconds()/(24*60*60)
df_adm['OUTPUT_LABEL'] = (df_adm.DAYS_NEXT_ADMIT < 30).astype('int')

In [8]:
df_adm = df_adm[df_adm['ADMISSION_TYPE']!='NEWBORN']
df_adm = df_adm[df_adm.DEATHTIME.isnull()]
df_adm['DURATION'] = (df_adm['DISCHTIME']-df_adm['ADMITTIME']).dt.total_seconds()/(24*60*60)

df_notes = pd.read_csv('/home/dc925/project/data/graphmimic/MIMIC/NOTEEVENTS.csv')
df_notes = df_notes.sort_values(by=['SUBJECT_ID','HADM_ID','CHARTDATE'])
df_adm_notes = pd.merge(df_adm[['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DAYS_NEXT_ADMIT','NEXT_ADMITTIME','ADMISSION_TYPE','DEATHTIME','OUTPUT_LABEL','DURATION']],
                        df_notes[['SUBJECT_ID','HADM_ID','CHARTDATE','TEXT','CATEGORY']], 
                        on = ['SUBJECT_ID','HADM_ID'],
                        how = 'left')

df_adm_notes.ADMITTIME_C = df_adm_notes.ADMITTIME.apply(lambda x: str(x).split(' ')[0])
df_adm_notes['ADMITTIME_C'] = pd.to_datetime(df_adm_notes.ADMITTIME_C, format = '%Y-%m-%d', errors = 'coerce')
df_adm_notes['CHARTDATE'] = pd.to_datetime(df_adm_notes.CHARTDATE, format = '%Y-%m-%d', errors = 'coerce')

/home/dc925/miniconda3/envs/graphmimic/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/dc925/miniconda3/envs/graphmimic/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  if sys.path[0] == '':


In [47]:
df_discharge = df_adm_notes[df_adm_notes['CATEGORY'] == 'Discharge summary']
# multiple discharge summary for one admission -> after examination -> replicated summary -> replace with the last one 
df_discharge = (df_discharge.groupby(['SUBJECT_ID','HADM_ID']).nth(-1)).reset_index()
df_discharge=df_discharge[df_discharge['TEXT'].notnull()]

In [48]:
df_discharge.OUTPUT_LABEL.value_counts()

0    40973
1     2907
Name: OUTPUT_LABEL, dtype: int64

In [49]:

import re
def preprocess1(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('admission date:','',y)
    y=re.sub('discharge date:','',y)
    y=re.sub('--|__|==','',y)
    return y

def preprocessing(df_less_n): 
    df_less_n['TEXT']=df_less_n['TEXT'].fillna(' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\n',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\r',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].apply(str.strip)
    df_less_n['TEXT']=df_less_n['TEXT'].str.lower()

    df_less_n['TEXT']=df_less_n['TEXT'].apply(lambda x: preprocess1(x))

    #to get chunks for readmission tasks
    from tqdm import tqdm
    df_len = len(df_less_n)
    want=pd.DataFrame({'HADM_ID':[],'TEXT':[],'OUTPUT_LABEL':[]})
    chunk_size = 500
    for i in tqdm(range(df_len)):
        x=df_less_n.TEXT.iloc[i].split()
        n=int(len(x)/chunk_size)
        if n < 1:
            want = want.append({'TEXT': df_less_n.TEXT.iloc[i], 'OUTPUT_LABEL':df_less_n.OUTPUT_LABEL.iloc[i],'HADM_ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
        else:
            for j in range(n):
                want=want.append({'TEXT':' '.join(x[j*chunk_size:(j+1)*chunk_size]),'OUTPUT_LABEL':df_less_n.OUTPUT_LABEL.iloc[i],'HADM_ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
            if len(x)%chunk_size>100:
                want=want.append({'TEXT':' '.join(x[-(len(x)%chunk_size):]),'OUTPUT_LABEL':df_less_n.OUTPUT_LABEL.iloc[i],'HADM_ID':df_less_n.HADM_ID.iloc[i]},ignore_index=True)
    
    return want

In [50]:
df_discharge = preprocessing(df_discharge)

100%|██████████| 43880/43880 [13:15<00:00, 55.17it/s] 


In [66]:
df_discharge

,HADM_ID,TEXT,OUTPUT_LABEL
0,145834.0,date of birth: sex: m service: medicine chief ...,0.0
1,145834.0,and his oxygen saturation was 100% on 2 liters...,0.0
2,145834.0,and a hyperdynamic right ventricle; suggesting...,0.0
3,145834.0,"of gentamicin on hospital day three, and his f...",0.0
4,145834.0,"p.o. q.d. (times one week), then 200 mg p.o. q...",0.0
...,...,...,...
139062,197084.0,po bid (2 times a day). calcium carbonate 200 ...,0.0
139063,137810.0,service: surgery allergies: zantac attending: ...,0.0
139064,137810.0,day. simvastatin 20 mg tablet sig: one (1) tab...,0.0
139065,113369.0,date of birth: sex: f service: neurosurgery al...,0.0


In [ ]:
# df_discharge = df_discharge.rename(columns={'ID':'HADM_ID'})
# df_discharge = df_discharge[["HADM_ID", "TEXT", "OUTPUT_LABEL"]]

In [ ]:
# df_discharge

In [52]:
# add icd codes
dicd = pd.read_csv('/home/dc925/project/data/graphmimic/MIMIC/DIAGNOSES_ICD.csv.gz')

gp = dicd.groupby('HADM_ID')
icd_codes = gp['ICD9_CODE'].apply(list)
icd_codes = icd_codes.reset_index()
df_all = df_discharge.merge(
    icd_codes,
    on='HADM_ID', how='left'
)

# Function to convert ICD9 codes to broader version (take the first 3 characters; first 4 characters for E-codes)
def simplify_icd(code_list):
    new_list = []
    for code in code_list:
        code = str(code)
        if code=='nan':
            continue
        if code[0]=='E':
            new_code = code[:4]
        else:
            new_code = code[:3]
        new_list.append(new_code)
    return new_list

df_all['BROAD_ICD9_CODE'] = df_all['ICD9_CODE'].apply(lambda x: simplify_icd(x))


In [73]:
from collections import Counter
broad_codes = df_all['BROAD_ICD9_CODE']
bc = list(broad_codes)
all_codes = [item for sublist in bc for item in sublist]
c = Counter(all_codes)
len(c)

1053

In [54]:
# Create a dictionary of broad ICD codes to descriptions
broad_codes_list = pd.read_csv('/home/dc925/project/data/graphmimic/UMLS/broad_codes.txt', sep='|', header=None)
broad_codes_list.columns = ["CUI", "ICD_BROAD", "DESC"]
broad_dict = broad_codes_list.set_index("ICD_BROAD").to_dict()['DESC']

In [55]:
# Check that all codes that we got from MIMIC are covered in the dictionary
all_codes_in_mimic = list(c.keys())
for code in all_codes_in_mimic:
    if code not in broad_dict:
        print(code)

In [56]:
pd.Series(all_codes_in_mimic).to_csv('/home/dc925/project/data/graphmimic/readmission/icd_codes_readmission.txt', index=None, header=None)

In [57]:
df_all['BROAD_ICD9_CODE'] = df_all['BROAD_ICD9_CODE'].apply(lambda x: ['ICD'+c for c in x])

In [58]:
df_all = df_all[["HADM_ID", "TEXT", "BROAD_ICD9_CODE", "OUTPUT_LABEL"]]

In [59]:
df_all

,HADM_ID,TEXT,BROAD_ICD9_CODE,OUTPUT_LABEL
0,145834.0,date of birth: sex: m service: medicine chief ...,"[ICD038, ICD785, ICD584, ICD427, ICD410, ICD42...",0.0
1,145834.0,and his oxygen saturation was 100% on 2 liters...,"[ICD038, ICD785, ICD584, ICD427, ICD410, ICD42...",0.0
2,145834.0,and a hyperdynamic right ventricle; suggesting...,"[ICD038, ICD785, ICD584, ICD427, ICD410, ICD42...",0.0
3,145834.0,"of gentamicin on hospital day three, and his f...","[ICD038, ICD785, ICD584, ICD427, ICD410, ICD42...",0.0
4,145834.0,"p.o. q.d. (times one week), then 200 mg p.o. q...","[ICD038, ICD785, ICD584, ICD427, ICD410, ICD42...",0.0
...,...,...,...,...
139062,197084.0,po bid (2 times a day). calcium carbonate 200 ...,"[ICD999, ICD568, ICD577, ICD285, ICD584, ICD57...",0.0
139063,137810.0,service: surgery allergies: zantac attending: ...,"[ICD441, ICD428, ICD998, ICD285, ICD424, ICD25...",0.0
139064,137810.0,day. simvastatin 20 mg tablet sig: one (1) tab...,"[ICD441, ICD428, ICD998, ICD285, ICD424, ICD25...",0.0
139065,113369.0,date of birth: sex: f service: neurosurgery al...,"[ICD756, ICD786, ICD401, ICD250, ICD413]",0.0


In [77]:
df_adm;

In [60]:
readmit_ID = df_adm[df_adm.OUTPUT_LABEL == 1].HADM_ID
not_readmit_ID = df_adm[df_adm.OUTPUT_LABEL == 0].HADM_ID
#subsampling to get the balanced pos/neg numbers of patients for each dataset
not_readmit_ID_use = not_readmit_ID.sample(n=len(readmit_ID), random_state=1)
id_val_test_t=readmit_ID.sample(frac=0.2,random_state=1)
id_val_test_f=not_readmit_ID_use.sample(frac=0.2,random_state=1)

id_train_t = readmit_ID.drop(id_val_test_t.index)
id_train_f = not_readmit_ID_use.drop(id_val_test_f.index)

id_val_t=id_val_test_t.sample(frac=0.5,random_state=1)
id_test_t=id_val_test_t.drop(id_val_t.index)

id_val_f=id_val_test_f.sample(frac=0.5,random_state=1)
id_test_f=id_val_test_f.drop(id_val_f.index)

# test if there is overlap between train and test, should return "array([], dtype=int64)"
(pd.Index(id_test_t).intersection(pd.Index(id_train_t))).values

id_test = pd.concat([id_test_t, id_test_f])
test_id_label = pd.DataFrame(data = list(zip(id_test, [1]*len(id_test_t)+[0]*len(id_test_f))), columns = ['id','label'])

id_val = pd.concat([id_val_t, id_val_f])
val_id_label = pd.DataFrame(data = list(zip(id_val, [1]*len(id_val_t)+[0]*len(id_val_f))), columns = ['id','label'])

id_train = pd.concat([id_train_t, id_train_f])
train_id_label = pd.DataFrame(data = list(zip(id_train, [1]*len(id_train_t)+[0]*len(id_train_f))), columns = ['id','label'])

In [61]:
discharge_train = df_all[df_all.HADM_ID.isin(train_id_label.id)]
discharge_val = df_all[df_all.HADM_ID.isin(val_id_label.id)]
discharge_test = df_all[df_all.HADM_ID.isin(test_id_label.id)]

In [62]:
# subsampling for training....since we obtain training on patient admission level so now we have same number of pos/neg readmission
# but each admission is associated with different length of notes and we train on each chunks of notes, not on the admission, we need
# to balance the pos/neg chunks on training set. (val and test set are fine) Usually, positive admissions have longer notes, so we need 
# find some negative chunks of notes from not_readmit_ID that we haven't used yet

df = pd.concat([not_readmit_ID_use, not_readmit_ID])
df = df.drop_duplicates(keep=False)
#check to see if there are overlaps
(pd.Index(df).intersection(pd.Index(not_readmit_ID_use))).values

# for this set of split with random_state=1, we find we need 400 more negative training samples
not_readmit_ID_more = df.sample(n=400, random_state=1)
discharge_train_snippets = pd.concat([df_all[df_all.HADM_ID.isin(not_readmit_ID_more)], discharge_train])

#shuffle
discharge_train_snippets = discharge_train_snippets.sample(frac=1, random_state=1).reset_index(drop=True)

#check if balanced
discharge_train_snippets.OUTPUT_LABEL.value_counts()


1.0    8426
0.0    8411
Name: OUTPUT_LABEL, dtype: int64

In [68]:
discharge_train_snippets.groupby(['HADM_ID']).agg(len).mean()

TEXT               3.363364
BROAD_ICD9_CODE    3.363364
OUTPUT_LABEL       3.363364
dtype: float64

In [69]:
discharge_val.groupby(['HADM_ID']).agg(len).mean()

TEXT               3.373473
BROAD_ICD9_CODE    3.373473
OUTPUT_LABEL       3.373473
dtype: float64

In [71]:
discharge_test.groupby(['HADM_ID']).agg(len).mean()

TEXT               3.339041
BROAD_ICD9_CODE    3.339041
OUTPUT_LABEL       3.339041
dtype: float64

In [63]:
discharge_val.OUTPUT_LABEL.value_counts()

1.0    1030
0.0     903
Name: OUTPUT_LABEL, dtype: int64

In [64]:
discharge_test.OUTPUT_LABEL.value_counts()

1.0    1077
0.0     873
Name: OUTPUT_LABEL, dtype: int64

In [65]:
discharge_train_snippets.to_csv('/home/dc925/project/data/graphmimic/readmission/train.csv', index=False, sep='\t')
discharge_val.to_csv('/home/dc925/project/data/graphmimic/readmission/valid.csv', index=False, sep='\t')
discharge_test.to_csv('/home/dc925/project/data/graphmimic/readmission/test.csv', index=False, sep='\t')

In [136]:
discharge_train_snippets

,HADM_ID,TEXT,BROAD_ICD9_CODE,OUTPUT_LABEL
0,159009.0,hypovolemic. it is reassuring that he has resp...,"[ICD852, ICD200, ICD584, ICD458, ICD238, ICD18...",1.0
1,121758.0,insulin lispro 100 unit/ml solution sig: one (...,"[ICD038, ICD785, ICD585, ICD008, ICD558, ICD55...",1.0
2,132572.0,service: medicine allergies: patient recorded ...,"[ICD427, ICD511, ICD202, ICD427, ICD599, ICD99...",1.0
3,128318.0,date of birth: sex: f service: medicine allerg...,"[ICD531, ICD996, ICD205, ICD078, ICD287, ICD27...",1.0
4,115148.0,"right-sided opacity, in correlation with prior...","[ICD288, ICD204, ICD789, ICD276, ICD112, ICD78...",0.0
...,...,...,...,...
12561,148413.0,of aricept and namenda. high degree av block s...,"[ICD253, ICD348, ICD790, ICDV55, ICD041, ICD27...",0.0
12562,125288.0,date of birth: sex: f service: medicine allerg...,"[ICD403, ICD585, ICD284, ICD710, ICD789, ICD28...",1.0
12563,101870.0,"name: , unit no: date of birth: sex: m service...","[ICD287, ICD518, ICD428, ICD486, ICD790, ICD35...",1.0
12564,113651.0,po q6h:prn cough heparin 5000 unit sc tid hydr...,"[ICD719, ICD568, ICD785, ICD584, ICD038, ICD99...",0.0
